<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing an Agentic RAG App</h1>

In this tutorial, you will:

- Build an agentic RAG app
- Instrument and trace the agentic RAG app with Phoenix
- Inspect the trace data in Phoenix

## Background

Agentic RAG (Retrieval Augmented Generation) combines the power of traditional RAG systems with autonomous agents that can make decisions and take actions. While traditional RAG simply retrieves relevant context and generates responses, agentic RAG adds a layer of agency - the ability to:

- Break down complex queries into sub-tasks
- Choose appropriate tools and actions to complete those tasks
- Reason about and synthesize information from multiple sources
- Make decisions about what information is relevant

For example, rather than just answering questions about data directly, an agentic RAG system might:
1. Analyze the user's question to determine required information
2. Query multiple data sources or tools as needed
3. Combine and reason about the retrieved information
4. Generate a comprehensive response

This tutorial demonstrates building an agentic RAG system using LlamaIndex's ReAct agent framework combined with vector and SQL query tools.

Let's get started!

ℹ️ This notebook requires an OpenAI API key.

## Import Dependencies

In [ ]:
!pip install -q llama-index openai arize-phoenix-otel openinference-instrumentation-llama-index llama-index-vector-stores-chroma chromadb sqlalchemy

In [ ]:
import os
from getpass import getpass

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from llama_index.core import VectorStoreIndex
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.chroma import ChromaVectorStore
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

## Add Secret Keys

### OpenAI API Key

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

### Phoenix Key

Phoenix can be run locally or access via phoenix.arize.com for free.

In [ ]:
if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass("🔑 Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={phoenix_api_key}"

## Instrument Our Agent

Now you'll connect your notebook to Phoenix. This will allow you to trace the agent's actions and see the trace data in Phoenix.

Phoenix's automatic instrumentation will trace any calls to LlamaIndex, meaning you don't need to do any extra instrumentation work.

In [ ]:
# configure the Phoenix tracer
tracer_provider = register(
    project_name="agentic-rag-demo",
    endpoint="https://app.phoenix.arize.com/v1/traces",  # change this endpoint if you're running Phoenix locally
)

# Finish automatic instrumentation
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

## Build Query Engine Tools using Chroma

Now you're ready to create the two databases that your agent will use to answer questions. This will be done using Chroma, a vector database that will store the company policies and employees.

### Company Policies Database

In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ["OPENAI_API_KEY"], model_name="text-embedding-3-small"
)

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.get_or_create_collection(
    "agentic-rag-demo-company-policies", embedding_function=openai_ef
)

chroma_collection.add(
    ids=["1", "2", "3"],
    documents=[
        "The travel policy is: Employees must book travel through the company portal. Economy class flights and standard hotel rooms are covered. Meals during travel are reimbursed up to $75/day. All expenses require receipts.",
        "The pto policy is: Full-time employees receive 20 days of paid time off per year, accrued monthly. PTO requests must be submitted at least 2 weeks in advance through the HR portal. Unused PTO can carry over up to 5 days into the next year.",
        "The dress code is: Business casual attire is required in the office. This includes collared shirts, slacks or knee-length skirts, and closed-toe shoes. Jeans are permitted on Fridays. No athletic wear or overly casual clothing.",
    ],
)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

chroma_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
chroma_engine_policy = chroma_index.as_query_engine(similarity_top_k=1)

In [ ]:
chroma_collection.peek()

In [ ]:
chroma_engine_policy.query("What is the travel policy?")

### Employees Database

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.get_or_create_collection(
    "agentic-rag-demo-company-employees", embedding_function=openai_ef
)

chroma_collection.add(
    ids=["1", "2", "3"],
    documents=[
        "John Smith is a Software Engineer in the Engineering department who started on 2023-01-15",
        "Sarah Johnson is a Marketing Manager in the Marketing department who started on 2022-08-01",
        "Michael Williams is a Sales Director in the Sales department who started on 2021-03-22",
    ],
)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

chroma_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
chroma_engine_employees = chroma_index.as_query_engine(similarity_top_k=1)

In [ ]:
chroma_engine_employees.query("What is the name of the Sales Director?")

### Add as Tools

LlamaIndex's ReAct agent framework allows you to add tools to the agent. Here you'll add the two tools that will be used to answer questions.

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=chroma_engine_employees,
        metadata=ToolMetadata(
            name="ChromaEmployees",
            description=(
                "Provides information about an employee's department, start date, and name from a relational database."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=chroma_engine_policy,
        metadata=ToolMetadata(
            name="ChromaPolicy",
            description=(
                "Provides information about company policies and preocedures. Use this to get more detailed information about company policies."
                "Use a detailed plain text statement about a specific policy as input to the tool."
            ),
        ),
    ),
]

## Create ReAct Agent

LlamaIndex provides a ReAct agent framework that allows you to create an agent that can use tools to answer questions. Here you'll create an agent that can use the two tools you created earlier to answer questions.

In [ ]:
CONTEXT = """
You are a chatbot designed to answer questions about the company's employees and policies.
You have access to a Chroma database with information about the company's employees and their departmental information.
You also have access to a Chroma database with information about the company's policies. Use provided context to help answer
the question. Make sure that you have all the context required to answer the question and if you don't, check if there are
other tools that can help you answer the question. If you still can't answer the question, ask the user for more information and
apologize that you can't answer.
"""

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True, context=CONTEXT)

## Test it out!

In [ ]:
response = agent.chat("What department is Sarah in?")
print(str(response))

In [ ]:
response = agent.chat("What is the pto policy for the ML Solutions team?")
print(str(response))